In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 16
refRatio = 2

deriv = 'CD'
order = 6

L = 1.
locs = [0.65]
epsilons = [1, 0.5]# 11.68]
mus = [1, 0.5]#0.99837]

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh_min / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max

physics = BT.PhysProps(omega, epsilons, mus, locs, L)
matInd = matInd = 8 # physics.matInd

xNode = omega.xNode
xCell = omega.xCell
dx = omega.h
print(np.arange(degFreed))
print(xNode)
print(dx)
print('')
print(xCell[matInd])
print(degFreed - matInd - 2)
print(matInd)
print(order)
print('')



distance is
 [ 0.65     0.61875  0.5875   0.55625  0.525    0.49375  0.4625   0.43125  0.4      0.36875  0.3375   0.30625  0.275    0.24375  0.2125   0.18125  0.15     0.0875   0.025   -0.0375  -0.1     -0.1625  -0.225   -0.2875  -0.35   ] 

distance is
 [1.      0.96875 0.9375  0.90625 0.875   0.84375 0.8125  0.78125 0.75    0.71875 0.6875  0.65625 0.625   0.59375 0.5625  0.53125 0.5     0.4375  0.375   0.3125  0.25    0.1875  0.125   0.0625  0.     ] 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[0.      0.03125 0.0625  0.09375 0.125   0.15625 0.1875  0.21875 0.25    0.28125 0.3125  0.34375 0.375   0.40625 0.4375  0.46875 0.5     0.5625  0.625   0.6875  0.75    0.8125  0.875   0.9375  1.     ]
[0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625 ]

0.265625
14
8
6



In [4]:
def MakeSpaceDeriv(omega, order, diff, matInd0 = -1):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = OT.CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = int(order + 1)
        else:
            orderStenc = order
        off = ((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            stenc = OT.UDStencil(order)
            loBound = -off / 2.
            hiBound = (off - 1.) / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = OT.DDStencil(order)
                off = int(off - 1)
                loBound = -off / 2.
                hiBound = (off + 1.) / 2.
                print('loBound:', loBound)
                print('hiBound:', hiBound)
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    
    print('stenc:', stenc)
    
#     stenc = np.ones(orderStenc + 1)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    # Index before fine-coarse interface
    p = np.where(spots > 0)[0][0]
    # Index before coarse-fine interface
    q = np.where(spots < 0)[0][0]
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)
    print(cellFaces)
    print('')
    
    

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    # CHANGE MADE HERE!
    
    print('p is', p)
    print('q is', q)
    print('matInd0 is', matInd0)
    
    if (matInd0 >= 0):
        if ((order >= matInd0) or (order > degFreed - matInd0 - 2)):
            errorMess = 'order is too high for given patch boundary and material boundary locations!'
        else:
            materialOverwrite = True
            matIndVec = [matInd0, degFreed - 1]
    else:
        materialOverwrite = False
        matIndVec = []
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    n_c_max = abs(off)
    for i in range(orderStenc):
        print('iter:', i)
        polyStencSet[i], n_c, n_f = GTT.CentGhost(omega, order, cellFaces[i])
        for matInd in (matIndVec):
            if ((abs(p - matInd) < n_c) and (cellFaces[i] > 0) and (n_c > n_c_max)):
                n_c_max = n_c
                print('LOOK OVER HERE!!!')
                print('n_c:', n_c)
                print('matInd:', matInd)
                print('cellFaces[i]:', cellFaces[i])
#                 polyStencSet[i] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                print('')
    
    polyStencSet = np.asarray(polyStencSet)
    print(polyStencSet)
    print('')
    
    
    # END CHANGE MADE!
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        print('')
        print('BEGIN ITERATION ' + str(s))
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0

        if (s > 0):
            j = int(off - s)
            pAt = p
            pLow = (p - 1) % degFreed
            pHi = (p + 1) % degFreed
            qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
            for i in range(s):
#                 print('j =', j)
                polyMat[pAt, :] = 0
                polyMat[pAt, pLow:pHi] = 0.5
                polyMat[qAt, :] = polyStencSet[j, :]
                pAt = (pAt - 1) % degFreed
                pLow = (pLow - 2) % degFreed
                pHi = (pHi - 2) % degFreed
                qAt = (qAt + 1) % degFreed
                j = int(j + 1)
                
            # CHANGE MADE HERE!
            
            if (materialOverwrite):
                for matInd in matIndVec:
                    print('h:', matInd)
                    if ((matInd <= p) and (p - matInd <= s)):
                        print('FIRST CASE:')
                        print('total rows =', s + 1)
                        for i in range(matInd-s+1, matInd+2): # (matInd+s+1, matInd+(2*s)+2):
                            j = i % degFreed
                            print(i, j, 'range:', matInd-s+1, matInd+1) # matInd+s+1, matInd+(2*s)+1)
                            polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                        print('')
                    else:
                        if ((matInd <= q) and (q - matInd <= s)):
                            print('SECOND CASE:')
                            print('total rows =', q - matInd + s)
                            for i in range(matInd-s+1, q+1): # (matInd+s+1, q+(2*s)+1):
                                j = i % degFreed
                                print(i, j, 'range:', matInd-s+1, q) # matInd+s+1, q+(2*s))
                                polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                            print('')
                        else:
                            print('THIRD CASE:')
                            print('total rows =', s)
                            for i in range(matInd-s+1, matInd+1): # (matInd+s+1, matInd+(2*s)+1):
                                j = i % degFreed
                                print(i, j, 'range:', matInd-s+1, matInd) # matInd+s+1, matInd+(2*s))
                                polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                            print('')
                    
            # END CHANGE MADE!
        
        if (s < 0):
            j = int(off) # - s - 1
            qAt = (q + 1) % degFreed
            qLow = (q + 1) % degFreed
            qHi = (q + 3) % degFreed
            pAt = (p + 1) % degFreed#p
            for i in range(abs(s)):
#                 print('j =', j)
#                 print('pAt is', pAt)
                polyMat[qAt, :] = 0
                polyMat[qAt, qLow:qHi] = 0.5
                polyMat[pAt, :] = polyStencSet[j, :]
#                 print(polyMat[pAt, :])
#                 print('')
                qAt = (qAt + 1) % degFreed
                qLow = (qLow + 2) % degFreed
                qHi = (qHi + 2) % degFreed
                pAt = (pAt + 1) % degFreed
                j = int(j + 1) # - 1
            
            # CHANGE MADE HERE!
            
            if (materialOverwrite):
                for matInd in matIndVec:
                    print('h:', matInd)
                    if ((matInd >= p) and (matInd - p <= abs(s))):
                        print('FIRST CASE:')
                        print('total rows =', matInd - p - s)
                        for i in range((p+1), (matInd-s+1)): # (p+(2*s)+1, matInd+s+1):
                            j = i % degFreed
                            print(i, 'range:', p+1, matInd-s) # p+(2*s)+1, matInd+s)
                            print(i % degFreed, 'alt range:', (p+1) % degFreed, (matInd-s) % degFreed)
                            polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                        print('')
                    else:
                        print('SECOND CASE:')
                        print('total rows =', abs(s))
                        for i in range((matInd+1), (matInd-s+1)): # (matInd+(2*s)+1, matInd+s+1):
                            j = i % degFreed
                            print(i, j, 'range:', matInd+1, matInd-s) # matInd+(2*s)+1, matInd+s)
                            print(i % degFreed, 'alt range:', (matInd+1) % degFreed, (matInd-s) % degFreed)
                            polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s)
                            print('row is:', polyMat[j, :])
                        print('')
                        if ((matInd < p) and (p - matInd <= n_c_max)):
                            print('SECOND ADD-ON CASE:')
                            print('total rows =', p - matInd)
                            for i in range((matInd-s+1), (p-s+1)): # (matInd+s+1, p+s+1):
                                j = i % degFreed
                                print(i, j, 'range:', matInd-s+1, p-s) # matInd+s+1, p+s)
                                print(i % degFreed, 'alt range:', (matInd-s+1) % degFreed, (p-s) % degFreed)
                                polyMat[j, :] = GTT.CentGhostMaterial(omega, order, matInd, i+s, s, revBounds = True)
                            print('')

                # END CHANGE MADE!
        
        rfunc = np.vectorize(round) 
        np.set_printoptions(suppress=True)
        print('A^(' + str(s) + ') =')
        print(rfunc(polyMat, 2))
        print('')
        matThis = derivMat @ polyMat
#         print(polyMat)
#         print('')
        print('D^(' + str(s) + ') =')
        print(rfunc(matThis, 2))
        print('')
        derivOp = derivOp + matThis
        print('sum to D^(' + str(s) + ') =')
        print(rfunc(derivOp, 2))
        print('')
        print('END ITERATION ' + str(s))
        print('')
        
    
    hMat = OT.StepMatrix(omega)
    
    derivOp = hMat @ derivOp
        
    return derivOp

In [5]:
A = MakeSpaceDeriv(omega, order, deriv, matInd)

print(A)
Asum = np.round(sum(A, axis = 1), 10)
print(Asum)
indices = np.where(Asum != 0)[0]
print(indices)
print(Asum[9])
print('')

stenc: [-0.01666667  0.15       -0.75        0.          0.75       -0.15        0.01666667]
[-1.5 -1.  -0.5  0.5  1.   1.5]

p is 15
q is 23
matInd0 is 8
iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
[[-0.47632576  0.23795996 -0.04329004  0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.00568182 -0.06872294  0.73484848  0.60984848]
 [-0.85416667  0.57440476 -0.20684524  0.03125     0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.         -0.00297619  0.04166667  1.41666667]
 [ 0.85416667 -0.57440476  0.20684524 -0.03125     0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.      

polyInterp is [  28.         -189.00000003  540.00000007 -840.00000009  756.00000007 -378.00000003   84.        ]
row is: [   0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.           28.         -189.00000003  540.00000007 -840.00000009  756.00000007 -378.00000003   84.        ]

A^(-3) =
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e+00 -7.000e+00  2.100e+01 -3.500e+01  3.500e+01 -2.100e+01  7.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  7.000e+00 -4.800e+01  1.400e+02 -2.240e+02  2.100e+02 -1.120e+02  2.800e+

In [6]:

print(A)
print('')
print(B)
print('')

[[-7.84000000e+01  1.92000000e+02 -2.40000000e+02  2.13333333e+02 -1.20000000e+02  3.84000000e+01 -5.33333333e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.33333333e+00 -4.10666667e+01  8.00000000e+01 -5.33333333e+01  2.66666667e+01 -8.00000000e+00  1.06666667e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.06666667e+00 -1.28000000e+01 -1.86666667e+01  4.26666667e+01 -1.60000000e+01  4.26666667e+00 -5.33333333e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 

NameError: name 'B' is not defined

In [ ]:
A = np.zeros((4, 4), float)
A[:, 0] = 1

A[0, 1] = - 3. / 2.
A[1, 1] = - 1. / 2.
A[2, 1] = 1. / 2.
A[3, 1] = 3. / 2.

A[0, 2] = 7. / 3.
A[1, 2] = 1. / 3.
A[2, 2] = 1. / 3.
A[3, 2] = 7. / 3.

A[0, 3] = - 15. / 4.
A[1, 3] = - 1. / 4.
A[2, 3] = 1. / 4.
A[3, 3] = 15. / 4.

B = LA.inv(A)

In [ ]:
print(A)
print('')
print(B)
print('')
print(sum(B, axis = 0))

In [ ]:
A = np.asarray([-84., 378., -756., 841., -540., 189., -28.])
B = np.asarray([28., -112., 209., -224., 140., -48., 7.])
C = np.asarray([-7., 22., -35., 35., -21., 7., -1.])
print(sum((0.01666667 * A) + (0.15 * B) + (0.75 * C)))


In [11]:
A = np.asarray([-2.45, 6., -7.5, 6.66666667, -3.75, 1.2, -0.16666667])
B = A / 0.03125
C = np.asarray([-78.4, 192.00000001, -240.00000002, 213.33333335, -120.00000002, 38.40000001, -5.33333333])
D = np.asarray([-7.84000000e+01, 1.92000000e+02, -2.40000000e+02, 2.13333333e+02, -1.20000000e+02, 3.84000000e+01, -5.33333333e+00])
print(A)
print(B)
print(sum(B))
print(sum(C))
print(sum(D))
print('')

[-2.45        6.         -7.5         6.66666667 -3.75        1.2        -0.16666667]
[ -78.4         192.         -240.          213.33333344 -120.           38.4          -5.33333344]
-1.687538997430238e-14
1.5987211554602254e-14
-3.2999999710625616e-07

